# Lab 3: Advanced Agent Patterns - SOLUTIONS

**Module 3 - Advanced Agent Development**

| Duration | Difficulty | Framework | Exercises |
|----------|------------|-----------|----------|
| 120 min | Advanced | LangChain | 3 |

## Setup

In [ ]:
import os
import json
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage, SystemMessage

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

llm = ChatOpenAI(model="gpt-4", temperature=0.3)

---

## Exercise 1: Chain-of-Thought Prompting - SOLUTION

In [ ]:
def create_cot_prompt(question: str) -> str:
    """Create a Chain-of-Thought prompt."""
    prompt = f"""You are a helpful assistant that solves problems step by step.

Question: {question}

Let's solve this step by step:

Step 1: First, let me identify what we need to find.

Step 2: Now, let me break down the information given.

Step 3: Let me work through the calculations or reasoning.

Step 4: Finally, let me combine everything for the answer.

Please show your complete reasoning for each step, then provide the final answer.
"""
    return prompt


def solve_with_cot(question: str) -> dict:
    """Solve a problem using Chain-of-Thought."""
    prompt = create_cot_prompt(question)
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return {
        "question": question,
        "reasoning": response.content,
    }

In [ ]:
# Test problems
problems = [
    "A store has 45 apples. They sell 12 in the morning and receive a shipment of 30. How many apples do they have now?",
    "If a train travels at 60 mph for 2 hours, then 80 mph for 1.5 hours, what's the total distance?",
    "A rectangle has a perimeter of 24 cm. If the length is twice the width, what are the dimensions?"
]

for problem in problems:
    result = solve_with_cot(problem)
    print(f"\n{'='*60}")
    print(f"Problem: {problem}")
    print(f"\nReasoning:\n{result['reasoning']}")

---

## Exercise 2: Plan-and-Execute Agent - SOLUTION

In [ ]:
class PlanAndExecuteAgent:
    def __init__(self, llm):
        self.llm = llm
        self.plan = []
        self.results = []
    
    def create_plan(self, task: str) -> list:
        """Create a step-by-step plan for the task."""
        planning_prompt = f"""
You are a planning assistant. Create a detailed step-by-step plan for this task.
Return ONLY a numbered list (1., 2., 3., etc.) with 3-6 clear, actionable steps.

Task: {task}

Plan:
"""
        
        response = self.llm.invoke([HumanMessage(content=planning_prompt)])
        
        # Parse numbered list into steps
        lines = response.content.strip().split('\n')
        self.plan = []
        for line in lines:
            # Match lines starting with numbers
            line = line.strip()
            if line and (line[0].isdigit() or line.startswith('-')):
                # Remove numbering
                clean_line = re.sub(r'^[\d]+[.\)]\s*', '', line)
                clean_line = re.sub(r'^-\s*', '', clean_line)
                if clean_line:
                    self.plan.append(clean_line)
        
        return self.plan
    
    def execute_step(self, step: str, context: str) -> str:
        """Execute a single step of the plan."""
        execution_prompt = f"""
You are executing a step in a larger plan.

Previous context and results:
{context if context else "(This is the first step)"}

Current step to execute: {step}

Execute this step thoroughly and provide the result:
"""
        response = self.llm.invoke([HumanMessage(content=execution_prompt)])
        return response.content
    
    def run(self, task: str) -> dict:
        """Run the full plan-and-execute cycle."""
        # Reset state
        self.plan = []
        self.results = []
        
        # Phase 1: Planning
        print("Creating plan...")
        plan = self.create_plan(task)
        print(f"Plan created with {len(plan)} steps:")
        for i, step in enumerate(plan):
            print(f"  {i+1}. {step}")
        
        # Phase 2: Execution
        print("\n" + "="*50)
        print("EXECUTION PHASE")
        print("="*50)
        
        context = ""
        for i, step in enumerate(plan):
            print(f"\n--- Executing Step {i+1} ---")
            print(f"Step: {step}")
            result = self.execute_step(step, context)
            self.results.append({"step": step, "result": result})
            context += f"\n\nStep {i+1} ({step}):\n{result}"
            print(f"Result: {result[:200]}..." if len(result) > 200 else f"Result: {result}")
        
        return {
            "task": task,
            "plan": plan,
            "results": self.results
        }

In [ ]:
# Test the agent
agent = PlanAndExecuteAgent(llm)
result = agent.run("Research and summarize the key differences between BERT and GPT models")

---

## Exercise 3: Self-Reflecting Agent - SOLUTION

In [ ]:
class ReflectiveAgent:
    def __init__(self, llm, max_iterations=3):
        self.llm = llm
        self.max_iterations = max_iterations
    
    def generate(self, task: str) -> str:
        """Generate initial response."""
        prompt = f"Complete this task thoroughly:\n\n{task}"
        response = self.llm.invoke([HumanMessage(content=prompt)])
        return response.content
    
    def critique(self, task: str, response: str) -> dict:
        """Critique the response and suggest improvements."""
        critique_prompt = f"""
You are a critical reviewer. Analyze this response to a task.

Task: {task}

Response to critique:
{response}

Provide a structured critique with:
1. Score (1-10, where 10 is perfect)
2. Strengths (list 2-3)
3. Weaknesses (list 2-3)
4. Specific improvements needed (list 2-3 actionable items)

Return ONLY valid JSON in this exact format:
{{"score": 7, "strengths": ["str1", "str2"], "weaknesses": ["weak1", "weak2"], "improvements": ["imp1", "imp2"]}}
"""
        result = self.llm.invoke([HumanMessage(content=critique_prompt)])
        
        try:
            # Try to extract JSON from the response
            content = result.content
            # Find JSON in the response
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                return json.loads(json_match.group())
            return json.loads(content)
        except:
            return {"score": 5, "strengths": [], "weaknesses": ["Parse error"], "improvements": ["Could not parse critique"]}
    
    def improve(self, task: str, response: str, critique: dict) -> str:
        """Improve response based on critique."""
        improve_prompt = f"""
You previously wrote a response that received feedback. Now write an improved version.

Original Task: {task}

Your Previous Response:
{response}

Feedback Received:
- Score: {critique.get('score', 'N/A')}/10
- Weaknesses identified: {critique.get('weaknesses', [])}
- Improvements needed: {critique.get('improvements', [])}

Write a significantly improved response that addresses ALL the feedback:
"""
        result = self.llm.invoke([HumanMessage(content=improve_prompt)])
        return result.content
    
    def run(self, task: str) -> dict:
        """Run the reflection loop."""
        print(f"Task: {task}\n")
        print("="*50)
        print("ITERATION 0: Initial Generation")
        print("="*50)
        
        response = self.generate(task)
        print(f"Response: {response[:300]}...\n" if len(response) > 300 else f"Response: {response}\n")
        
        history = [{"iteration": 0, "response": response}]
        
        for i in range(self.max_iterations):
            print(f"\n{'='*50}")
            print(f"ITERATION {i+1}: Critique & Improve")
            print("="*50)
            
            critique = self.critique(task, response)
            print(f"Score: {critique.get('score', 'N/A')}/10")
            print(f"Strengths: {critique.get('strengths', [])}")
            print(f"Weaknesses: {critique.get('weaknesses', [])}")
            print(f"Improvements: {critique.get('improvements', [])}")
            
            if critique.get('score', 0) >= 8:
                print("\nQuality threshold met! Stopping iterations.")
                break
            
            print("\nGenerating improved response...")
            response = self.improve(task, response, critique)
            print(f"Improved: {response[:300]}..." if len(response) > 300 else f"Improved: {response}")
            
            history.append({
                "iteration": i + 1,
                "critique": critique,
                "response": response
            })
        
        return {"final_response": response, "history": history}

In [ ]:
# Test the reflective agent
agent = ReflectiveAgent(llm, max_iterations=3)
result = agent.run("Write a concise explanation of how neural networks learn")

print("\n" + "="*50)
print("FINAL RESULT")
print("="*50)
print(result['final_response'])

---

## Checkpoint

Congratulations! You've completed Lab 3 with solutions. Key takeaways:

- **Chain-of-Thought**: Explicit step prompts improve reasoning
- **Plan-and-Execute**: Separating planning from execution gives better control
- **Self-Reflection**: Critique loops can iteratively improve output quality

**Next:** Lab 4 - RAG Pipeline Implementation